In [20]:
import pandas as pd
import pickle

In [21]:
with open("model/model.pkl", "rb") as f:
    model = pickle.load(f)
model

RandomForestRegressor(criterion='absolute_error', max_depth=10,
                      max_features='sqrt', min_samples_leaf=4, n_estimators=300,
                      random_state=31415)

In [22]:
cvs_mapping = pd.read_parquet("data/CVs_predict/CVs_mapping.parquet")
for col in cvs_mapping.columns:
    cvs_mapping[col] = cvs_mapping[col].str.replace(".pdf", "")
cvs_mapping

,orig_filename,new_filename
0,Cerys Okandeji,CVANON0000
1,Emma Natalie Smallwood,CVANON0001
2,Pablo Valero,CVANON0002
3,Kaleigh Alyssa Arza,CVANON0003
4,Eva Thorland Gallegos,CVANON0004
...,...,...
76,Ciara Fitzgerald,CVANON0076
77,Isobella Kristina Fortnam Tomlinson,CVANON0077
78,Samantha Clowes,CVANON0078
79,Jordon Campbell,CVANON0079


In [23]:
df_predict = pd.read_parquet("data/df_predict_cleaned.parquet")
df_predict

,id,professionality,visual,grammar,ai_likelihood,education,experience,languages
0,CVANON0002,-3.251596,-1.383111,-0.865650,-2.387977,-0.670991,-0.753450,0.169880
1,CVANON0009,-3.251596,-1.383111,-0.220131,-1.518529,0.032415,-0.753450,0.169880
2,CVANON0033,-0.534280,-0.389286,1.070907,-0.649081,-0.248947,-0.600124,0.169880
3,CVANON0010,-0.534280,-0.389286,-0.220131,1.089815,0.032415,-0.743228,-0.697506
4,CVANON0015,-3.251596,-1.383111,-2.156688,1.959263,-0.952353,-0.753450,-0.263813
...,...,...,...,...,...,...,...,...
76,CVANON0018,1.277264,0.604539,0.425388,0.220367,2.142632,0.209945,0.820419
77,CVANON0019,-0.534280,-0.389286,-1.511169,-0.649081,0.454458,-0.076262,-0.046967
78,CVANON0070,-0.534280,-0.389286,-0.865650,0.220367,-0.670991,-0.589902,-0.263813
79,CVANON0017,0.371492,-0.389286,-1.511169,-0.649081,0.173096,-0.030265,0.603573


In [24]:
predictions = df_predict["id"].copy()
predictions = pd.merge(predictions, cvs_mapping, left_on="id", right_on="new_filename", how="inner")
predictions = predictions.drop(columns=["new_filename"])
predictions = predictions.rename(columns={"orig_filename": "name"})
predictions

,id,name
0,CVANON0002,Pablo Valero
1,CVANON0009,Lyra Fenech
2,CVANON0033,Samuel Burgr
3,CVANON0010,Lara López Groskreutz
4,CVANON0015,Ian Velasquez
...,...,...
76,CVANON0018,Anoushka Barbara Grant
77,CVANON0019,Siobhan O'Mahoney
78,CVANON0070,Carmen Roberts
79,CVANON0017,DANIELA TRUJILLO LAMBERTI


In [25]:
predictions["points_9m"] = model.predict(df_predict.iloc[:, 1:])
predictions

,id,name,points_9m
0,CVANON0002,Pablo Valero,1148.085459
1,CVANON0009,Lyra Fenech,1242.775054
2,CVANON0033,Samuel Burgr,1037.734929
3,CVANON0010,Lara López Groskreutz,885.599570
4,CVANON0015,Ian Velasquez,1028.812722
...,...,...,...
76,CVANON0018,Anoushka Barbara Grant,893.927058
77,CVANON0019,Siobhan O'Mahoney,1600.149152
78,CVANON0070,Carmen Roberts,1125.594366
79,CVANON0017,DANIELA TRUJILLO LAMBERTI,1300.346608


In [26]:
predictions.to_parquet("predictions/predictions.parquet")
predictions.to_excel("predictions/predictions.xlsx")